## MicroPython on an ESP32

The following is a collection of notes on the installation and the 
usage of MicroPython on an ESP32, here tested using an **Adafruit Huzzah32**. Note that while the Huzzah32 was created by Adafruit, their
fork of MicroPython ("CircuitPython") is not yet available for 
this board.

_Acknowledgements:_ These notes heavely relied on the following earlier, very helpful resources:
- Installation:  
  - https://lemariva.com/blog/2017/10/micropython-getting-started  
  - https://github.com/pvanallen/esp32-getstarted
- Jupyter notebook for MicroPython:
  - https://github.com/goatchurchprime/jupyter_micropython_kernel/

### Setting up MicroPython on the ESP32 under Windows 10:

1. Install USB to Serial driver for the ESP32:  
    [Download driver here](https://www.silabs.com/products/development-tools/software/usb-to-uart-bridge-vcp-drivers)
    
    
2. Restart computer


3. Update pip and install some tools:
   ```
   python -m pip install --upgrade pip
   pip install msgpack
   pip install esptool
   pip install adafruit-ampy
   ```
   
4. Download the latest Micropython software binary for ESP32:  
    [Download firmware here](https://micropython.org/download/#esp32)
  
  
5. Connect the device to the PC using an USB cable, and check the serial
   interface number where the ESP32 is connected (e.g. COM5).


6. In a (power)shell, erase the flash of the ESP32, with `<xx>` indicating
   the current COM port and `<yyy>` the name of the firmware file.
   ```
   python -m esptool --port COM<xx> erase_flash
   python -m esptool --port COM<xx> --baud 460800 write_flash --flash_mode  dio --flash_size=detect 0x1000 <yyy>.bin
   ```
   
7. Now you should be ready to go. Start a terminal program (e.g. PuTTY) and 
   connect to the COM port (at 115200 BAUD). You should see the chevrons 
   (`>>>`) of the [REPL prompt](https://docs.micropython.org/en/latest/esp8266/esp8266/tutorial/repl.html),
   indicating that the device is ready. Try something like 
   `print("Hello")` to see if the device responds.
   

8. In a powershell (or bash), use the tool `ampy` (in a powershell or bash)
   to manipulate files on the ESP32. See https://github.com/adafruit/ampy for details.
   For example, copy a python file to the flash of the ESP32 using:
   ```
   ampy --port COM<xx> put <pythonfilename>.py
   ```
   List the files in the ESP32's flash memory:
   ```
   ampy --port COM<xx> ls
   ```
   View a file in the flash:
   ```
   ampy --port COM<xx> get <filename>
   ```
   Remove a file from the flash:
   ```
   ampy --port COM<xx> rm <filename>
   ```
   Set default COM port to use shorter commands:
   ```
   $env:AMPY_PORT = 'COM<xx>'
   ampy ls
   ```

   _Note:_ In principle, similar commands ae also available from a jupyter 
   notebook connected to a python kernel running on the device (see below, 
   and see `%lsmagic` for commands), but I could not get them to work reliably.


### Using Atom to interact with MicroPython on an ESP32

The `pymakr` plugin for [Atom](https://atom.io/) allows to interact with a 
ESP32 via the REPL. In addition, one can also use it to upload all files of 
a project (in a specified folder) or run individual `.py` files directly.


### Using Jupyter to interact with a ESP32 over its serial REPL

The following instructions assume that Jupyter is already installed, 
preferentially [Anaconda](https://www.anaconda.com/distribution/). This is a summary 
of the instructions in the respective github repository 
[goatchurchprime/jupyter_micropython_kernel](https://github.com/goatchurchprime/jupyter_micropython_kernel/)
by Julian Todd; please see there for further details and updates.

1. Clone this repository to a directory (e.g. `MicroPython`) using:
   ```
   git clone https://github.com/goatchurchprime/jupyter_micropython_kernel.git
   ```


2. Install this library (in editable mode) into Python3 using:
   ```
   pip install -e jupyter_micropython_kernel
   ```
   This creates a small file pointing to this directory in the `python/../site-packages directory`, 
   and makes it possible to `git update` the library later as it gets improved. 
 
 
3. Install the kernel into jupyter itself using:
   ```
   python -m jupyter_micropython_kernel.install
   ```
   This creates the small file `.local/share/jupyter/kernels/micropython/kernel.json` 
   that jupyter uses to reference its kernels.


4. Now run Jupyter notebooks:
   ```
   jupyter notebook
   ```
   
   
5. All "magic" commands can be listed with `%lsmagic`. The most important
   one connects the notebook to the MicroPython REPL on the device:

In [2]:
%serialconnect

Connecting to --port=COM5 --baud=115200 
OSError: [Errno 2] ENOENT
MicroPython v1.9.4-512-g6f015d337 on 2018-09-11; ESP32 module with ESP32
Type "help()" for more information.
>>>[reboot detected 0]repl is in normal command mode
[\r\x03\x03] b'\r\n>>> '
[\r\x01] b'\r\n>>> \r\nraw REPL; CTRL-B to exit\r\n>' Ready.


### Running some tests

Switching red onboard LED on ...

In [3]:
import machine

pin = machine.Pin(13, machine.Pin.OUT)
pin.value(1)

... and off:

In [4]:
pin.value(0)

Modulate the LED sinusoidally:

In [14]:
import math
import time

# Initialise PWM for pin 13
#
pwm = machine.PWM(machine.Pin(13))

# Change the duty cycle (0..1023) sinusoidally, `nCycle` times
#
nSteps  = 50
nCycles = 10
for iC in range(nCycles):
    for iS in range(nSteps):
        pwm.duty(int((math.sin(iS/nSteps *2 *math.pi) +1) /2 *1023))
        time.sleep(0.01)

# Release pin        
#
pwm.deinit()

.

### More resources

- https://github.com/lemariva/ESP32MicroPython
- https://github.com/pvanallen/esp32-getstarted
  (Useful starting up info)
- https://github.com/espressif/esptool
- https://learn.adafruit.com/micropython-hardware-ili9341-tft-and-featherwing/micropython
- https://github.com/loboris/MicroPython_ESP32_psRAM_LoBo/wiki
- https://medium.com/@rovai/micropython-on-esp-using-jupyter-6f366ff5ed9


## MicroPython on an ESP8266

Install micropython, e.g. Wemos D1 Pro (4 MB):
```   
python -m esptool --port COM<xx> erase_flash
python -m esptool --port COM<xx> --baud 115200 write_flash -fm dio --flash_size=4MB 0 <yyy>.bin
```